<img align="left" src="https://ithaka-labs.s3.amazonaws.com/static-files/images/tdm/tdmdocs/CC_BY.png"><br />

Created by [Nathan Kelber](http://nkelber.com) and Ted Lawless for [JSTOR Labs](https://labs.jstor.org/) under [Creative Commons CC BY License](https://creativecommons.org/licenses/by/4.0/)<br />
For questions/comments/improvements, email nathan.kelber@ithaka.org.<br />
___

**Finding Significant Words Using TF/IDF**

**Description:**
Discover the significant words in a corpus using Gensim TF-IDF. The following code is included:

* Filtering based on a pre-processed ID list
* Filtering based on a stop words list
* Token cleaning
* Computing TF-IDF using Gensim

**Use Case:** For Researchers (Mostly code without explanation, not ideal for learners)

**Difficulty:** Intermediate

**Completion time:** 5-10 minutes

**Knowledge Required:** 
* Python Basics Series ([Start Python Basics I](./python-basics-1.ipynb))

**Knowledge Recommended:**
* [Exploring Metadata](./metadata.ipynb)
* [Working with Dataset Files](./working-with-dataset-files.ipynb)
* [Pandas I](./pandas-1.ipynb)
* [Creating a Stopwords List](./creating-stopwords-list.ipynb)
* A familiarity with [gensim](https://docs.tdm-pilot.org/key-terms/#gensim) is helpful but not required.

**Data Format:** [JSON Lines (.jsonl)](https://docs.tdm-pilot.org/key-terms/#jsonl)

**Libraries Used:**
* `pandas` to load a preprocessing list
* `csv` to load a custom stopwords list
* [gensim](https://docs.tdm-pilot.org/key-terms/#gensim) to help compute the [tf-idf](https://docs.tdm-pilot.org/key-terms/#tf-idf) calculations
* [NLTK](https://docs.tdm-pilot.org/key-terms/#nltk) to create a stopwords list (if no list is supplied)

**Research Pipeline:**

1. Build a dataset
2. Create a "Pre-Processing CSV" with [Exploring Metadata](./exploring-metadata.ipynb) (Optional)
3. Create a "Custom Stopwords List" with [Creating a Stopwords List](./creating-stopwords-list.ipynb) (Optional)
4. Complete the TF-IDF analysis with this notebook
____

# Import Raw Dataset

In [ ]:
# Creating a variable `dataset_id` to hold our dataset ID
# The default dataset is Shakespeare Quarterly, 1950-present
dataset_id = "7e41317e-740f-e86a-4729-20dab492e925"

# Pull in the dataset that matches `dataset_id`
# in the form of a gzipped JSON lines file.
import tdm_client
dataset_file = tdm_client.get_dataset(dataset_id)

# Load Pre-Processing Filter (Optional)
If you completed pre-processing with the "Exploring Metadata and Pre-processing" notebook, you can use your CSV file of dataset IDs to automatically filter the dataset. 

In [ ]:
# Import a pre-processed CSV file of filtered dataset IDs.
# If you do not have a pre-processed CSV file, the analysis
# will run on the full dataset and may take longer to complete.
import pandas as pd
import os

pre_processed_file_name = f'data/pre-processed_{dataset_id}.csv'

if os.path.exists(pre_processed_file_name):
    df = pd.read_csv(pre_processed_file_name)
    filtered_id_list = df["id"].tolist()
    use_filtered_list = True
    print('Pre-Processed CSV found. Successfully read in ' + str(len(df)) + ' documents.')
else: 
    use_filtered_list = False
    print('No pre-processed CSV file found. Full dataset will be used.')

# Load Stop Words List (Optional)
The default stop words list is NLTK. You can also create a stopwords CSV with the "Creating Stop Words" notebook.

In [ ]:
# Load a custom data/stop_words.csv if available
# Otherwise, load the nltk stopwords list in English

# Create an empty Python list to hold the stopwords
stop_words = []

# The filename of the custom data/stop_words.csv file
stopwords_list_filename = 'data/stop_words.csv'

if os.path.exists(stopwords_list_filename):
    import csv
    with open(stopwords_list_filename, 'r') as f:
        stop_words = list(csv.reader(f))[0]
    print('Custom stopwords list loaded from CSV')
else:
    # Load the NLTK stopwords list
    from nltk.corpus import stopwords
    stop_words = stopwords.words('english')
    print('NLTK stopwords list loaded')

# Define a Unigram Cleaning Function
By default, this function will:

* Lowercase all tokens
* Remove tokens in stopwords list
* Remove tokens with fewer than 4 characters
* Remove tokens with non-alphabetic characters

In [ ]:
# Define a function that will process individual tokens

def process_token(token):
    token = token.lower()
    if token in stop_words: # Remove stop tokens
        return
    if len(token) < 4: # Remove short tokens
        return
    if not(token.isalpha()): # Remove non-alphanumeric tokens
        return
    return token

# Process and Collect Unigrams

In [ ]:
# Collecting the unigrams and processing them into `documents`

limit = 500 # Change number of documents being analyzed. Set to `None` to do all documents.
n = 0
documents = []
document_ids = []
    
for document in tdm_client.dataset_reader(dataset_file):
    processed_document = []
    document_id = document['id']
    if use_filtered_list is True:
        # Skip documents not in our filtered_id_list
        if document_id not in filtered_id_list:
            continue
    document_ids.append(document_id)
    unigrams = document.get("unigramCount", [])
    for gram, count in unigrams.items():
        clean_gram = process_token(gram)
        if clean_gram is None:
            continue
        processed_document.append(clean_gram)
    if len(processed_document) > 0:
        documents.append(processed_document)
    n += 1
    if (limit is not None) and (n >= limit):
        break
print('Unigrams collected and processed.')

---
# Using Gensim to Compute "Term Frequency- Inverse Document Frequency"

It will be helpful to remember the basic steps we did in the explanatory [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) example:

1. Create a list of the frequency of every word in every document
2. Create a list of every word in the [corpus](https://docs.tdm-pilot.org/key-terms/#corpus)
3. Compute [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) based on that data

So far, we have completed the first item by creating a list of the frequency of every word in every document. Now we need to create a list of every word in the corpus. In [gensim](https://docs.tdm-pilot.org/key-terms/#gensim), this is called a "dictionary". A [gensim dictionary](https://docs.tdm-pilot.org/key-terms/#gensim-dictionary) is similar to a [Python dictionary](https://docs.tdm-pilot.org/key-terms/#python-dictionary), but here it is called a [gensim dictionary](https://docs.tdm-pilot.org/key-terms/#gensim-dictionary) to show it is a specialized kind of dictionary.

## Creating a Gensim Dictionary

Let's create our [gensim dictionary](https://docs.tdm-pilot.org/key-terms/#gensim-dictionary). A [gensim dictionary](https://docs.tdm-pilot.org/key-terms/#gensim-dictionary) is a kind of masterlist of all the words across all the documents in our corpus. Each unique word is assigned an ID in the gensim dictionary. The result is a set of key/value pairs of unique tokens and their unique IDs.

In [ ]:
import gensim
dictionary = gensim.corpora.Dictionary(documents)

Now that we have a [gensim dictionary](https://docs.tdm-pilot.org/key-terms/#gensim-dictionary), we can get a preview that displays the number of unique tokens across all of our texts.

In [ ]:
print(dictionary)

The [gensim dictionary](https://docs.tdm-pilot.org/key-terms/#gensim-dictionary) stores a unique identifier (starting with 0) for every unique token in the corpus. The [gensim dictionary](https://docs.tdm-pilot.org/key-terms/#gensim-dictionary) does not contain information on word frequencies; it only catalogs all the words in the corpus. You can see the unique ID for each token in the text using the .token2id() method. Your corpus may have hundreds of thousands of unique words so here we just give a preview of the first ten.

In [ ]:
dict(list(dictionary.token2id.items())[0:10]) # Print the first ten tokens and their associated IDs.

We can also look up the corresponding ID for a token using the ``.get`` method.

In [ ]:
dictionary.token2id.get('people', 0) # Get the value for the key 'people'. Return 0 if there is no token matching 'people'. The number returned is the gensim dictionary ID for the token. 

## Creating a Bag of Words Corpus


### Example: A Single Document

The next step is to combine our word frequency data found within ``documents`` to our [gensim dictionary](https://docs.tdm-pilot.org/key-terms/#gensim-dictionary) token IDs. For every document, we want to know how many times a word (notated by its ID) occurs. We can do a single document first to show how this works. We will create a [Python list](https://docs.tdm-pilot.org/key-terms/#python-list) called ``example_bow_corpus`` that will turn our word counts into a series of [tuples](https://docs.tdm-pilot.org/key-terms/#tuple) where the first number is the [gensim dictionary](https://docs.tdm-pilot.org/key-terms/#gensim-dictionary) token ID and the second number is the word frequency.

In [ ]:
example_bow_corpus = [dictionary.doc2bow(documents[0])] # Create an example bag of words corpus. We select a document at random to use as our sample.
list(example_bow_corpus[0][:10]) # List out the first ten tuples in ``example_bow_corpus``

Using IDs can seem a little abstract, but we can discover the word associated with a particular ID. For demonstration purposes, the following code will replace the token IDs in the last example with the actual tokens.

In [ ]:
word_counts = [[(dictionary[id], count) for id, count in line] for line in example_bow_corpus]
list(word_counts[0][:10])

We saw before that you could discover the [gensim dictionary](https://docs.tdm-pilot.org/key-terms/#gensim-dictionary) ID number by running:

> dictionary.token2id.get('people', 0)

If you wanted to discover the token given only the ID number, the method is a little more involved. You could use [list comprehension](https://docs.tdm-pilot.org/key-terms/#list-comprehensions) to find the **key** token based on the **value** ID. Normally, [Python dictionaries](https://docs.tdm-pilot.org/key-terms/#python-dictionary) only map from keys to values (not from values to keys). However, we can write a quick [list comprehension](https://docs.tdm-pilot.org/key-terms/#list-comprehensions) to go the other direction. (It is unlikely one would ever do these methods in practice, but they are shown here to demonstrate how the [gensim dictionary](https://docs.tdm-pilot.org/key-terms/#gensim-dictionary) is connected to the list entries in the [gensim](https://docs.tdm-pilot.org/key-terms/#gensim) ``bow_corpus``. 

In [ ]:
[token for dict_id, token in dictionary.items() if dict_id == 100] # Find the corresponding token in our gensim dictionary for the gensim dictionary ID

### Example: All Documents

We have seen an example that demonstrates how the [gensim](https://docs.tdm-pilot.org/key-terms/#gensim) [bag of words](https://docs.tdm-pilot.org/key-terms/#bag-of-words) [corpus](https://docs.tdm-pilot.org/key-terms/#corpus) works on a single document. Let's apply it now to all of our documents. 

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in documents]
#print(bow_corpus[:3]) #Show the bag of words corpus for the first 3 documents

The next step is to create the [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) model which will set the parameters for our implementation of [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf). In our [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) example, the formula for [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) was:

$$(Times-the-word-occurs-in-given-document) \cdot \mbox{log} \frac{(Total-number-of-documents)}{(Number-of-documents-containing-word)}$$

In [gensim](https://docs.tdm-pilot.org/key-terms/#gensim), the default formula for measuring [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) uses log base 2 instead of log base 10, as shown:

$$(Times-the-word-occurs-in-given-document) \cdot \log_{2} \frac{(Total-number-of-documents)}{(Number-of-documents-containing-the-word)}$$

If you would like to use a different formula for your [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) calculation, there is a description of [parameters you can pass](https://radimrehurek.com/gensim/models/tfidfmodel.html).

## Create the `TfidfModel`

In [ ]:
model = gensim.models.TfidfModel(bow_corpus) # Create our gensim TF-IDF model

Now, we apply our model to the ``bow_corpus`` to create our results in ``corpus_tfidf``. The ``corpus_tfidf`` is a python list of each document similar to ``bow_document``. Instead of listing the frequency next to the [gensim dictionary](https://docs.tdm-pilot.org/key-terms/#gensim-dictionary) ID, however, it contains the TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) score for the associated token. Below, we display the first document in ``corpus_tfidf``. 

In [ ]:
corpus_tfidf = model[bow_corpus] # Create TF-IDF scores for the ``bow_corpus`` using our model
list(corpus_tfidf[0][:10]) # List out the TF-IDF scores for the first 10 tokens of the first text in the corpus

Let's display the tokens instead of the [gensim dictionary](https://docs.tdm-pilot.org/key-terms/#gensim-dictionary) IDs.

In [ ]:
example_tfidf_scores = [[(dictionary[id], count) for id, count in line] for line in corpus_tfidf]
list(example_tfidf_scores[0][:10]) # List out the TF-IDF scores for the first 10 tokens of the first text in the corpus

Finally, let's sort the terms by their [TF-IDF](https://docs.tdm-pilot.org/key-terms/#tf-idf) weights to find the most significant terms in the document.

In [ ]:
# Sort the tuples in our tf-idf scores list

def Sort(tfidf_tuples): 
    tfidf_tuples.sort(key = lambda x: x[1], reverse=True) 
    return tfidf_tuples 

list(Sort(example_tfidf_scores[0])[:10]) #List the top ten tokens in our example document by their TF-IDF scores

We could also analyze across the entire corpus to find the most unique terms. These are terms that appear frequently in a single text, but rarely or never appear in other texts. (Often, these will be proper names since a particular article may mention a name often but the name may rarely appear in other articles.)

In [ ]:
td = { # Define a dictionary ``td`` where each document gather
        dictionary.get(_id): value for doc in corpus_tfidf
        for _id, value in doc
    }
sorted_td = sorted(td.items(), key=lambda kv: kv[1], reverse=True) # Sort the items of ``td`` into a new variable ``sorted_td``, the ``reverse`` starts from highest to lowest

In [ ]:
for term, weight in sorted_td[:25]: # Print the top 25 terms in the entire corpus
    print(term, weight)

And, finally, we can see the most significant term in every document.

In [ ]:
# For each document, print the ID, most common word, and TF/IDF score

for n, doc in enumerate(corpus_tfidf):
    if len(doc) < 1:
        continue
    word_id, score = max(doc, key=lambda x: x[1])
    print(document_ids[n], dictionary.get(word_id), score)
    if n >= 10:
        break